In [2]:
train_path=r"C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\train.csv"
test_path=r'C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\test.csv'

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
import pandas as pd
train_df=pd.read_csv(train_path)
test_df=pd.read_csv(test_path)

In [5]:
train_df.head()

,review,aspect,sentiment
0,But the staff was so horrible to us .,staff,negative
1,"To be completely fair , the only redeeming fac...",food,positive
2,"The food is uniformly exceptional , with a ver...",food,positive
3,"The food is uniformly exceptional , with a ver...",kitchen,positive
4,"The food is uniformly exceptional , with a ver...",menu,neutral


In [6]:
train_df[train_df['sentiment']=='negative'].index

Index([   0,   14,   15,   16,   18,   29,   30,   39,   49,   94,
       ...
       3543, 3549, 3550, 3563, 3564, 3569, 3580, 3582, 3585, 3590],
      dtype='int64', length=807)

In [7]:
train_df[train_df['sentiment']=='negative']['review'][14]

'They did not have mayonnaise , forgot our toast , left out ingredients ( ie cheese in an omelet ) , below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it .'

In [8]:
train_df.value_counts('sentiment')

sentiment
positive    2164
negative     807
neutral      637
Name: count, dtype: int64

In [9]:
test_df.value_counts("sentiment")

sentiment
positive    727
negative    196
neutral     196
Name: count, dtype: int64

In [10]:
train_df.value_counts('aspect')

aspect
food                        340
service                     189
place                        59
menu                         56
prices                       56
                           ... 
Halibut                       1
Indian Fast Food              1
waiting area                  1
Indian dining experience      1
, chicken                     1
Name: count, Length: 1322, dtype: int64

# Data

In [11]:
index = 0
example = train_df.iloc[index]

print('Review:', example['review'])
print('Aspect:', example['aspect'])
print('Sentiment:', example['sentiment'])

Review: But the staff was so horrible to us .
Aspect: staff
Sentiment: negative


In [12]:
# Normalization of the text
import re

text = example['review']
# lowercase
text = text.lower()
# remove punctuations, special characters
text = re.sub(r"[^a-z0-9\s]", '', text)#means a-z and 0-9 egula chara onno kono value(like punchuation) thake segulo replace kore dibo empty("") string diye."^" sign ta bujay j a-z and 0-9 egula chara bakigula bad diye and \s mane bujay j space ta thakuk
# remove extra whitespace
text = ' '.join(text.split())
print('Review:', text)

Review: but the staff was so horrible to us


In [13]:
# Normalize the text
def normalize(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = ' '.join(text.split())
    return text

text = example['review']
normalized_text = normalize(text)
print(normalized_text)

but the staff was so horrible to us


In [14]:
""" Tokenization
Definition: Token is any sequential part of a text that collectively make up the entire text

Text = "but the staff was so horrible to us"

Word-level Tokens: "but", "the", "staff", "was", "so", "horrible", "to", "us"
Character level tokens: b,u,t,' ',t,h,e,' ',s,t,a,f,f,' ',w,a,s,' ',s,o, ' ',h,o,r,r,i,b,l,e,' ',t,o,' ',u,s
Subword: but, the, sta, ff, was, so, horr, ible, to us

For example: text = but the staff was so horrible to us
Tokens: "but", "the", "staff", "was", "so", "horrible", "to", "use" (word level tokenization)
Standard tokenization technique: Character-level tokenization
For example: text = "Life is good"
Character level tokens: 'L', 'i', 'f', 'e', ' ', 'g', 'o', 'o', 'd'
Advanced tokenization technique: BPE (Byte-Pair Encoding)
For example: text = "life is good"
Bypte pair tokens: li, fe, 'is', 'goo', 'd' #basically means subword tokanization
"""

def tokenize(text):
    tokens = text.split()
    return tokens

tokens = tokenize(normalized_text)
print(tokens)

['but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us']


In [15]:
#prottekta token ekta string but model receives only number.so each token is set to an number
"""suppose your `train_df['review']` contains:

texts = [
    "I love pizza",
    "I love burgers",
    "Pizza is tasty"
]

Initial vocabulary:
token_2_id = {
    "<PAD>": 0,
    "<UNK>": 1
}
idx = 2

PROCESS:

Text 1: "I love pizza"
Tokens: ["i", "love", "pizza"]

i → add as 2
love → add as 3
pizza → add as 4

Text 2: "I love burgers"
Tokens: ["i", "love", "burgers"]

i → already exists
love → already exists
burgers → add as 5

Text 3: "Pizza is tasty"
Tokens: ["pizza", "is", "tasty"]

pizza → exists
is → add as 6
tasty → add as 7

FINAL VOCABULARY:
{
  "<PAD>": 0,
  "<UNK>": 1,
  "i": 2,
  "love": 3,
  "pizza": 4,
  "burgers": 5,
  "is": 6,
  "tasty": 7
}

Vocabulary size = 8
"""
def build_vocab(texts):
    token_2_id = {
        '<PAD>': 0,
        '<UNK>': 1,
    }

    idx = 2
    for text in texts:
        normalized_text = normalize(text)
        tokens = tokenize(normalized_text)

        for token in tokens:
            if token_2_id.get(token) is None:
                token_2_id[token] = idx
                idx += 1
    return token_2_id

token_2_id = build_vocab(train_df['review'])
print("Vocabulary size:", len(token_2_id))

Vocabulary size: 3800


In [16]:
def convert_tokens_2_ids(tokens):
    input_ids = [
        token_2_id.get(token, token_2_id['<UNK>']) for token in tokens
    ]
    return input_ids

text = example['review'] + " hello "
normalized_text = normalize(text)
tokens = tokenize(normalized_text)
input_ids = convert_tokens_2_ids(tokens)

print("Vocabulary size:", len(token_2_id))
print(tokens)
print(input_ids)

Vocabulary size: 3800
['but', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', 'hello']
[2, 3, 4, 5, 6, 7, 8, 9, 1]


In [17]:
import pickle
with open("vocab.pkl", "wb") as f:#Opens a file named vocab.pkl in write-binary mode and assigns it to f.
    pickle.dump(token_2_id, f)#Saves the object token_2_id into the file using pickle.
label_map = {
    "negative": 0,
    "neutral": 1,
    "positive": 2,
}

In [18]:
from torch.utils.data import Dataset
import torch



class ABSADataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):#automatically called by dataloader
        example = train_df.iloc[idx]
        text = example['review']
        aspect = example['aspect']
        sentiment = example['sentiment']

        text_aspect_pair = text + ' ' + aspect
        normalized_text = normalize(text_aspect_pair)
        tokens = tokenize(normalized_text)
        input_ids = convert_tokens_2_ids(tokens)
        label_id = label_map[sentiment]
        return {
            "input_ids": input_ids,
            "label": label_id
        }


# Batch: Is a collection of examples
# Typically 32, 64, 128 etc.
# In a batch all the examples shape must match
# pixels, label

In [19]:
"""batch_input_ids = [ #batch size number ta dataloader bole dilei hobe..baki kaj ta dataloader nije handle kore nibe
    [5, 7, 9],
    [4, 2],
    [8, 3, 6, 1]
]

batch_labels = [0, 1, 0]

Step 2: Find max_len
Lengths → 3, 2, 4
max_len = 4

Step 3: Pad with pad_token_id = 0

[5, 7, 9]     → [5, 7, 9, 0]
[4, 2]        → [4, 2, 0, 0]
[8, 3, 6, 1]  → [8, 3, 6, 1]

batch_padded_input_ids = [
    [5, 7, 9, 0],
    [4, 2, 0, 0],
    [8, 3, 6, 1]
]

Step 4: Convert to tensors

batch_input_ids tensor:
tensor([
    [5, 7, 9, 0],
    [4, 2, 0, 0],
    [8, 3, 6, 1]
])

batch_labels tensor:
tensor([0, 1, 0])

Final output:
{
    "batch_input_ids": tensor([[5, 7, 9, 0],
                               [4, 2, 0, 0],
                               [8, 3, 6, 1]]),
    "batch_labels": tensor([0, 1, 0])
}"""
@staticmethod
def collate_fn(batch): #when to use this collab function ? when each batch size is different
    # batch size i.e. 32
    # [{input_ids: [], label: 0}, ....{input_ids: [], label: 1}]
    batch_input_ids = [item['input_ids'] for item in batch]
    batch_labels = [item['label'] for item in batch]

    max_len = max(len(input_ids) for input_ids in batch_input_ids)
    pad_token_id = token_2_id['<PAD>']
    batch_padded_input_ids = [
            input_ids + [pad_token_id] * (max_len - len(input_ids)) for input_ids in batch_input_ids
    ]

    return {
        "batch_input_ids": torch.tensor(
            batch_padded_input_ids,
            dtype=torch.long
        ),
        "batch_labels": torch.tensor(
            batch_labels,
            dtype=torch.long
        )
    }

In [23]:
train_ds=ABSADataset(train_df)
train_ds.__getitem__(3)

{'input_ids': [3,
  16,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  17,
  38,
  39,
  40,
  23,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  3,
  50,
  51,
  52,
  37],
 'label': 2}

In [79]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader


class ABSADataModule(pl.LightningDataModule):#ekhane dataset class and dataloader same clss er under e niye ansi
    def __init__(self, train_path, test_path, batch_size):
        super().__init__()
        self.train_path = train_path
        self.test_path = test_path
        self.batch_size = batch_size

    def setup(self, stage=None):#ei function ta ekdom surute auto call hoy.Suppose ,dataset ta cloud theke download korar lagte pare or dataset ta training er age onno kono kaj korar lagte par
        # prepare the dataset for training
        # You may download, process some stuff
        # Everything data related we need to do before training
        # Read the dataset
        train_df = pd.read_csv(self.train_path)
        test_df = pd.read_csv(self.test_path)

        # build vocabulary
        self.token_2_id = build_vocab(train_df['review'])
        # Initialize the dataset
        self.train_set = ABSADataset(train_df)
        self.test_set = ABSADataset(test_df)

    def train_dataloader(self):
        return DataLoader(
            self.train_set,
            batch_size=self.batch_size,
            shuffle=True,#THAT means prottekta epoch er surute batch er value gulo shuffle hobe
            collate_fn=self.collate_fn
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_set,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self.collate_fn
        )


    @staticmethod
    def collate_fn(batch):
        # batch size i.e. 32
        # [{input_ids: [], label: 0}, ....{input_ids: [], label: 1}]
        batch_input_ids = [item['input_ids'] for item in batch]
        batch_labels = [item['label'] for item in batch]

        max_len = max(len(input_ids) for input_ids in batch_input_ids)
        pad_token_id = token_2_id['<PAD>']
        batch_padded_input_ids = [
                input_ids + [pad_token_id] * (max_len - len(input_ids)) for input_ids in batch_input_ids
        ]

        return {
            "batch_input_ids": torch.tensor(
                batch_padded_input_ids,
                dtype=torch.long
            ),
            "batch_labels": torch.tensor(
                batch_labels,
                dtype=torch.long
            )
        }


In [80]:
data_module=ABSADataModule(
    train_path=train_path,
    test_path=test_path,
    batch_size=32,
)

# **Model**

In [81]:
from torch import optim
from torchmetrics.classification import Accuracy
import torch.nn as nn
import numpy as np

# class ABSA(nn.Module):
#     def __init__(self, vocab_size, num_labels=3):
#         super(ABSA, self).__init__()
#         self.vocab_size = vocab_size
#         self.num_labels = num_labels
#         self.embedding_layer = nn.Embedding(
#             num_embeddings=vocab_size, embedding_dim=256
#         )
#         # Sequence to sequence learning
#         self.lstm_layer = nn.LSTM(
#             input_size=256,
#             hidden_size=512,
#             batch_first=True, # as batch is first dimension in the input shape
#         )
#
#         self.fc_layer = nn.Linear(
#             in_features=512,
#             out_features=self.num_labels
#         )
#     def forward(self, x):
#         # input: (B, L)
#         embeddings = self.embedding_layer(x) # (B, L, 256)
#         lstm_out, _ = self.lstm_layer(embeddings) # (B, L, 512)
#         logits = self.fc_layer(lstm_out[:, -1, :])
#         return logits

class ABSAModel(pl.LightningModule):
    def __init__(self, vocab_size, num_labels=3):
        super().__init__()
        self.vocab_size = vocab_size
        self.num_labels = num_labels
        """Natural Language Processing e tokenization er pore  sobcheye significant layer hocche embedding layer.Embedding layer hocche ekta technique jekhane amra prottekta token k ekta boro list of numbers diye represent kori """
        # Embedding: Converts each token id into a vector of numbers
        # For example embedding size = 256
        # token: apple => 15 => [14.5, 12.5, ....., 1.25] of shape 256
        # [token_id][vector] => vocab_size x embedding dimension
        # For example: vocab_size=10, embedding_dim=5
        # [0] = [......]
        # [1] = [......]
        # .....
        # Example: I eat apple and orange.
        # Consider apple and orange
        #  # input ids shape = (B, 3,)
        # After that, shape = (B, L, 256)
        """"why we use embedding
        I love pizza" → [2, 5, 8]

        carry **no meaning**.
        ID 2 is not more similar to ID 5 — they are just labels.

        So we use **embeddings**.


        # ⭐ WHAT IS AN EMBEDDING?

        An embedding converts each word/tok into a **dense vector**:

        pizza → [0.21, -0.11, 0.87, ...]
        burger → [0.19, -0.09, 0.92, ...]
        """
        self.embedding_layer = nn.Embedding(
            num_embeddings=self.vocab_size,#means kotogula token er jonno embedding korte hobe
            embedding_dim=256
        )
        """tensor([[ 0.12, -0.34,  0.56,  0.78.....256 ta hobe],   # "I"
        [ 0.09,  0.44, -0.21,  0.87.....256 ta hobe],              # "love"
        [-0.11,  0.65,  0.32, -0.47.....256 ta hobe]])             # "pizza"
"""
        #sequence to sequence learning.means textual learning that means text theke kichu sikhte chacche.Modern architechture e transformer layer ta amra use kori
        self.lstm_layer = nn.LSTM(
            input_size=256,#cause ager layer theke 256 size asbe
            hidden_size=512,#typically output size ta input size er 2 times hoy
            batch_first=True, #jehetu amader processing e(B, L, 256) batch ta sobar age ase tai true diye disi

        )
        self.fc_layer = nn.Linear(
            in_features=512,
            out_features=self.num_labels#here num_labels=3 that means  # negative, neutral, positive scores
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.save_hyperparameters()
    def forward(self, x):
        embeddings = self.embedding_layer(x)#X means number of batch.embedding output:(B,L,256)
        lstm_out, _ = self.lstm_layer(embeddings)#output:(B,L,512)..."_" means hidden state er output jeta amder dorkar nai but layer nijer dorkar
        logits = self.fc_layer(lstm_out[:,-1,:])    #lstm layer er output ta k fully connected layer e connect kore disi
        #logits output: sobgulo batch nicche,lenght er sudu last value ta nicche and then output 512 sobgulai provide kortese
        """Now the question is why we are taking -1(infact why the last value).ok lets see
        I eat rice
        L=3

        I=logits 1
        I eat=logits2
        I eat rice =logits3
        another example:
        sentence: I love this movie
        | Token   | Output vector |
        | ------- | ------------- |
        | I →     | h1            |
        | love →  | h2            |
        | this →  | h3            |
        | movie → | **h4**        |
        We use h4, because it contains information from all previous steps.

        """
        # self.model = ABSA(vocab_size, num_labels)
        #

        # self.save_hyperparameters()
        return logits

    def training_step(self, batch, batch_idx):#automatically called by lightning module in trainer.fit
        input_ids = batch['batch_input_ids']
        labels = batch['batch_labels']
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        logits = self.forward(input_ids) # Calls the forward method
        loss = self.loss_fn(logits, labels)
        self.log('train_loss', loss, prog_bar=True)
        acc = self.compute_metrics(logits, labels)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):#automatically called by lightning module in trainer.fit
        input_ids = batch['batch_input_ids']
        labels = batch['batch_labels']
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        logits = self.forward(input_ids) # Calls the forward method
        loss = self.loss_fn(logits, labels)
        self.log('test_loss', loss, prog_bar=True)
        acc = self.compute_metrics(logits, labels)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self) :#automatically called by lightning module in trainer.fit
        return optim.Adam(self.parameters(), lr=3e-4)

    def compute_metrics(self, logits, labels):
        logits = logits.detach().cpu()
        labels = labels.detach().cpu()
        preds = torch.argmax(logits, dim=1)
        accuracy = Accuracy(task="multiclass", num_classes=self.num_labels)
        return accuracy(preds, labels)

In [82]:
data_module.setup()
model=ABSAModel(
    vocab_size=len(data_module.token_2_id),
    num_labels=3
)

# **Training**

In [83]:
trainer=pl.Trainer(
    max_epochs=10,
)
trainer.fit(model,data_module)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | embedding_layer | Embedding        | 972 K  | train
1 | lstm_layer      | LSTM             | 1.6 M  | train
2 | fc_layer        | Linear           | 1.5 K  | train
3 | loss_fn         | CrossEntropyLoss | 0      | train
-------------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.205    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python31

Training: |                                                                                              | 0/?…

`Trainer.fit` stopped: `max_epochs=10` reached.


In [84]:
trainer.test(model,data_module)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |                                                                                               | 0/?…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8811438679695129     │
│         test_loss         │    0.3562406301498413     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.3562406301498413, 'test_acc': 0.8811438679695129}]

In [101]:
import torch

# ----- 1. Set model to evaluation -----
model.eval()

# ----- 2. Input sentence -----
sentence = train_df[train_df['sentiment']=='negative']['review'][ 3543]

# Tokenize
tokens = build_vocab(sentence)

# Convert tokens → IDs
input_ids = [
        token_2_id.get(token, token_2_id['<UNK>']) for token in tokens
    ]

# Pad

# Convert to batch tensor
input_tensor = torch.tensor([input_ids])

# ----- 3. Predict -----
with torch.no_grad():
    logits = model(input_tensor)
    predicted_label = torch.argmax(logits, dim=1)
print(logits)

print("Sentence:", sentence)
print("Predicted Label:", predicted_label)


tensor([[-0.8363, -1.0272,  1.6279]])
Sentence: Kind of a small place but I guess if they are not too busy might be able to fit a group or kids .
Predicted Label: tensor([2])
